# Step 3: Test, Train, Split, OneHotEncode, Model Function

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import datetime as dt

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from matplotlib import dates as mpl_dates


In [2]:
df = pd.read_csv('data/cleaned_data')

In [3]:
df

,date,officer_id,officer_age,officer_race,officer_gender,officer_squad,subject_id,subject_age,subject_race,subject_gender,stop_resolution,weapon_type,initial_call_type,call_type,arrest,frisk,precinct,sector,beat
0,2015-10-16,7500,31,Black or African American,M,SOUTH PCT 1ST W - ROBERT,-1,NaN,Asian,Male,Arrest,None,NaN,NaN,0,0,South,O,O2
1,2015-04-03,6973,38,White,M,NaN,-1,NaN,Black or African American,Male,Field Contact,None,NaN,NaN,0,0,NaN,NaN,NaN
2,2015-04-05,7402,42,White,M,NaN,-1,NaN,Black or African American,Male,Field Contact,None,NaN,NaN,0,0,NaN,NaN,NaN
3,2015-05-20,7591,30,Hispanic or Latino,M,WEST PCT 3RD W - MARY,-1,NaN,Other,Female,Field Contact,None,NaN,NaN,0,0,NaN,NaN,NaN
4,2015-05-20,7591,30,Hispanic or Latino,M,WEST PCT 3RD W - MARY,-1,NaN,White,Male,Field Contact,None,NaN,NaN,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46153,2019-07-17,8633,31,Black or African American,M,TRAINING - FIELD TRAINING SQUAD,25974371966,56 and Above,Black or African American,Male,Arrest,NaN,SHOPLIFT - THEFT,911,1,0,East,E,E2
46154,2019-09-22,8616,33,White,M,WEST PCT 3RD W - KING,25974371966,56 and Above,Black or African American,Male,Offense Report,NaN,BURG - COMM BURGLARY (INCLUDES SCHOOLS),ALARM CALL (NOT POLICE ALARM),0,0,NaN,NaN,NaN
46155,2019-11-08,7769,50,White,M,WEST PCT 1ST W - KING/QUEEN,25974371966,56 and Above,Black or African American,Male,Offense Report,NaN,PURSE SNATCH - ROBBERY,ONVIEW,0,0,West,K,K3
46156,2020-12-26,6799,44,Hispanic or Latino,M,WEST PCT 2ND W - SPECIAL BEATS,25974371966,56 and Above,Black or African American,Male,Field Contact,NaN,SHOPLIFT - THEFT,911,0,0,West,K,K1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46158 entries, 0 to 46157
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   date               46158 non-null  object
 1   officer_id         46158 non-null  int64 
 2   officer_age        46158 non-null  int64 
 3   officer_race       46158 non-null  object
 4   officer_gender     46158 non-null  object
 5   officer_squad      45613 non-null  object
 6   subject_id         46158 non-null  int64 
 7   subject_age        45114 non-null  object
 8   subject_race       46158 non-null  object
 9   subject_gender     46158 non-null  object
 10  stop_resolution    46158 non-null  object
 11  weapon_type        34275 non-null  object
 12  initial_call_type  33482 non-null  object
 13  call_type          33482 non-null  object
 14  arrest             46158 non-null  int64 
 15  frisk              46158 non-null  int64 
 16  precinct           36700 non-null  objec

In [5]:
#isolate columns wanted to compare with frisk
frisk_columns = ['officer_age', 'officer_race', 'officer_gender', 'subject_race', 'subject_gender']

In [6]:
#Establish X and y
y = df['frisk']
X = df[frisk_columns] 

#Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#Remove 'object'-type features from X
cont_features = [col for col in X.columns if X[col].dtype in [np.float64, np.int64]]

# Remove "object"-type features from X_train and X_test
X_train_cont = X_train.loc[:, cont_features]
X_test_cont = X_test.loc[:, cont_features]

In [7]:
print(y.shape)
print(X.shape)

(46158,)
(46158, 5)


In [8]:
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer

# Impute missing values with median using SimpleImputer
impute = SimpleImputer(strategy='median')
X_train_imputed = impute.fit_transform(X_train_cont)
X_test_imputed = impute.transform(X_test_cont)

#Fit the model
linreg = LinearRegression()
linreg.fit(X_train_imputed, y_train)

# Print R2 and MSE for training and test sets
print('Training r^2:', linreg.score(X_train_imputed, y_train))
print('Test r^2:', linreg.score(X_test_imputed, y_test))
print('Training MSE:', mean_squared_error(y_train, linreg.predict(X_train_imputed)))
print('Test MSE:', mean_squared_error(y_test, linreg.predict(X_test_imputed)))

Training r^2: 0.00043783345690728837
Test r^2: 0.00026787510429937544
Training MSE: 0.17637816677375295
Test MSE: 0.17609274376163317


In [9]:
#normalize our data
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

# Scale the train and test data
ss = StandardScaler()
X_train_imputed_scaled = ss.fit_transform(X_train_imputed)
X_test_imputed_scaled = ss.transform(X_test_imputed)

# Fit the model 
linreg_norm = LinearRegression()
linreg_norm.fit(X_train_imputed_scaled, y_train)

# Print R2 and MSE for training and test sets
print('Training r^2:', linreg_norm.score(X_train_imputed_scaled, y_train))
print('Test r^2:', linreg_norm.score(X_test_imputed_scaled, y_test))
print('Training MSE:', mean_squared_error(y_train, linreg_norm.predict(X_train_imputed_scaled)))
print('Test MSE:', mean_squared_error(y_test, linreg_norm.predict(X_test_imputed_scaled)))

Training r^2: 0.00043783345690728837
Test r^2: 0.00026787510429937544
Training MSE: 0.17637816677375295
Test MSE: 0.17609274376163317


In [10]:
# Create X_cat which contains only the categorical variables
features_cat = [col for col in X.columns if X[col].dtype in [np.object]]
X_train_cat = X_train.loc[:, features_cat]
X_test_cat = X_test.loc[:, features_cat]

# Fill missing values with the string 'missing'
X_train_cat.fillna(value='missing', inplace=True)
X_test_cat.fillna(value='missing', inplace=True)

In [11]:
from sklearn.preprocessing import OneHotEncoder

# OneHotEncode categorical variables
ohe = OneHotEncoder(handle_unknown='ignore')

# Transform training and test sets
X_train_ohe = ohe.fit_transform(X_train_cat)
X_test_ohe = ohe.transform(X_test_cat)

# Convert these columns into a DataFrame 
columns = ohe.get_feature_names(input_features=X_train_cat.columns)
cat_train_df = pd.DataFrame(X_train_ohe.todense(), columns=columns)
cat_test_df = pd.DataFrame(X_test_ohe.todense(), columns=columns)

In [12]:
#combining all the X_train categorial and continuous data
X_train_all = pd.concat([pd.DataFrame(X_train_imputed_scaled), cat_train_df], axis=1)
X_test_all = pd.concat([pd.DataFrame(X_test_imputed_scaled), cat_test_df], axis=1)

In [13]:
linreg_all = LinearRegression()
linreg_all.fit(X_train_all, y_train)

print('Training r^2:', linreg_all.score(X_train_all, y_train))
print('Test r^2:', linreg_all.score(X_test_all, y_test))
print('Training MSE:', mean_squared_error(y_train, linreg_all.predict(X_train_all)))
print('Test MSE:', mean_squared_error(y_test, linreg_all.predict(X_test_all)))

Training r^2: 0.02406971897060528
Test r^2: 0.02495668686568142
Training MSE: 0.17220819237513355
Test MSE: 0.17174405825377295


In [18]:
#Create a function within many Machine Learning Models

def models(X_train_all, y_train):
    
    #Using Logistic Regression Algorithm to the Training Set
    from sklearn.linear_model import LogisticRegression
    log = LogisticRegression(random_state = 0, max_iter = 50000)
    log.fit(X_train_all, y_train)
    
    
    #Using KNeighborsClassifier Method of neighbors class to use Nearest Neighbor algorithm
    from sklearn.neighbors import KNeighborsClassifier
    knn = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
    knn.fit(X_train_all, y_train)
    

    #Using SVC method of svm class to use Support Vector Machine Algorithm
    from sklearn.svm import SVC
    svc_lin = SVC(kernel = 'linear', random_state = 0)
    svc_lin.fit(X_train_all, y_train)
    
    #Using SVC method of svm class to use Kernel SVM Algorithm
    from sklearn.svm import SVC
    svc_rbf = SVC(kernel = 'rbf', random_state = 0)
    svc_rbf.fit(X_train_all, y_train)
    
    
    #Using GaussianNB method of naïve_bayes class to use Naïve Bayes Algorithm
    from sklearn.naive_bayes import GaussianNB
    gauss = GaussianNB()
    gauss.fit(X_train_all, y_train)
    

    #Using DecisionTreeClassifier of tree class to use Decision Tree Algorithm
    from sklearn.tree import DecisionTreeClassifier
    tree = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
    tree.fit(X_train_all, y_train)
    
    #Using RandomForestClassifier method of ensemble class to use Random Forest Classification algorithm
    from sklearn.ensemble import RandomForestClassifier
    forest = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
    forest.fit(X_train_all, y_train)
    
    print('[0]Logistic Regression Training Accuracy:', log.score(X_train_all, y_train))
    print('[1]K Nearest Neighbor Training Accuracy:', knn.score(X_train_all, y_train))
    print('[2]Support Vector Machine (Linear Classifier) Training Accuracy:', svc_lin.score(X_train_all, y_train))
    print('[3]Support Vector Machine (RBF Classifier) Training Accuracy:', svc_rbf.score(X_train_all, y_train))
    print('[4]Gaussian Naive Bayes Training Accuracy:', gauss.score(X_train_all, y_train))
    print('[5]Decision Tree Classifier Training Accuracy:', tree.score(X_train_all, y_train))
    print('[6]Random Forest Classifier Training Accuracy:', forest.score(X_train_all, y_train))
    
    return log, knn, svc_lin, svc_rbf, gauss, tree, forest

In [15]:
X_train_all

,0,officer_race_American Indian/Alaska Native,officer_race_Asian,officer_race_Black or African American,officer_race_Hispanic or Latino,officer_race_Nat Hawaiian/Oth Pac Islander,officer_race_Not Specified,officer_race_Two or More Races,officer_race_White,officer_gender_F,...,subject_race_Asian,subject_race_Black or African American,subject_race_Hispanic,subject_race_Multi-Racial,subject_race_Nat Hawaiian/Oth Pac Islander,subject_race_Other,subject_race_Unknown,subject_race_White,subject_gender_Female,subject_gender_Male
0,0.892669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,-0.656134,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,-0.179579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,-0.179579,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,0.773530,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36921,-0.298718,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
36922,0.535253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
36923,-1.251827,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
36924,-1.132689,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [16]:
y_train

28854    1
19324    0
23076    1
35170    0
37365    0
        ..
21243    0
45891    0
42613    1
43567    1
2732     1
Name: frisk, Length: 36926, dtype: int64

In [19]:
models(X_train_all, y_train)

[0]Logistic Regression Training Accuracy: 0.7711910307100688
[1]K Nearest Neighbor Training Accuracy: 0.7270216107891458
[2]Support Vector Machine (Linear Classifier) Training Accuracy: 0.7711910307100688
[3]Support Vector Machine (RBF Classifier) Training Accuracy: 0.7711910307100688
[4]Gaussian Naive Bayes Training Accuracy: 0.6615934571846396
[5]Decision Tree Classifier Training Accuracy: 0.7826463738287386
[6]Random Forest Classifier Training Accuracy: 0.781563126252505


(LogisticRegression(max_iter=50000, random_state=0),
 KNeighborsClassifier(),
 SVC(kernel='linear', random_state=0),
 SVC(random_state=0),
 GaussianNB(),
 DecisionTreeClassifier(criterion='entropy', random_state=0),
 RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0))